In [182]:
import pandas as pd


# Load files
consumption_df = pd.read_excel(r"C:\Users\aadit\OneDrive\Desktop\project files\new Consumption Data for Trainee Ver1.XLSX")
stock_df = pd.read_excel(r"C:\Users\aadit\OneDrive\Desktop\project files\new Stock Data for Trainee Ver1.XLSX")

# Clean column names
consumption_df.columns = consumption_df.columns.str.strip().str.lower().str.replace(' ', '_')
stock_df.columns = stock_df.columns.str.strip().str.lower().str.replace(' ', '_')

# Rename key columns
consumption_df.rename(columns={'qty_in_unit_of_entry': 'quantity_issued'}, inplace=True)
stock_df.rename(columns={'unrestricted': 'unrestricted_stock', 'pe': 'period'}, inplace=True)

# New: Define unit mapping dictionary for proper full names
unit_mapping = {
    'NO': 'Number',
    'ST': 'Set',
    'L': 'Liter',
    'M': 'Meter',
    'FT': 'Foot',
    'EA': 'Each',
    'KG': 'Kilogram',
    # Add more based on your data
}

# New: Extract most common unit per material from consumption data
material_units = consumption_df.groupby('material')['unit_of_entry'].agg(lambda x: x.mode()[0] if not x.empty else 'Unknown').reset_index()
material_units['unit'] = material_units['unit_of_entry'].map(unit_mapping).fillna('Unknown')

# New: Check for consistency (if >1 unique unit per material)
unit_consistency = consumption_df.groupby('material')['unit_of_entry'].nunique()
inconsistent = unit_consistency[unit_consistency > 1].index.tolist()
if inconsistent:
    print(f"Warning: Inconsistent units for materials: {inconsistent}. Using most common.")

print("Extracted Units Sample:\n", material_units.head())

# Convert dates
consumption_df['posting_date'] = pd.to_datetime(consumption_df['posting_date'], errors='coerce')
stock_df['stock_date'] = pd.to_datetime(stock_df['year'].astype(str) + '-' + stock_df['period'].astype(str) + '-01', errors='coerce')

# Aggregate consumption (positive quantities)
consumption_df['quantity_issued'] = consumption_df['quantity_issued'].abs()  # Make positive
consumption_agg = consumption_df.groupby(['material', 'posting_date'])['quantity_issued'].sum().reset_index()

# Aggregate stock
stock_agg = stock_df.sort_values('stock_date').groupby('material').last()[['unrestricted_stock', 'stock_date']].reset_index()

# Merge with units
merged_df = pd.merge(consumption_agg, stock_agg, on='material', how='left')
merged_df = pd.merge(merged_df, material_units[['material', 'unit']], on='material', how='left')
merged_df['unit'] = merged_df['unit'].fillna('Unknown')  # Handle missing

# Add Material Description (pull first description per material from consumption_df)
desc = consumption_df.groupby('material')['material_description'].first().reset_index()  # Assuming 'material_description' column exists
merged_df = merged_df.merge(desc, on='material', how='left')

# Add Current Stock (from stock_agg, already in merged_df as 'unrestricted_stock')
# Renaming for clarity
merged_df.rename(columns={'unrestricted_stock': 'current_stock'}, inplace=True)

# Add Last Consumed Date (max posting_date per material)
last_consumed = consumption_df.groupby('material')['posting_date'].max().reset_index().rename(columns={'posting_date': 'last_consumed_date'})
merged_df = merged_df.merge(last_consumed, on='material', how='left')

print("\n✅ Merged Data with Proper Units and New Columns:")
print(merged_df.head())


Extracted Units Sample:
       material unit_of_entry    unit
0  210000050.0            NO  Number
1  210000051.0            NO  Number
2  210000052.0            ST     Set
3  210000056.0            NO  Number
4  210000057.0            NO  Number

✅ Merged Data with Proper Units and New Columns:
      material posting_date  quantity_issued  current_stock stock_date  \
0  210000050.0   2020-03-29              1.0            1.0 2019-11-01   
1  210000051.0   2020-03-29              1.0            1.0 2019-11-01   
2  210000052.0   2020-03-29              1.0            1.0 2019-11-01   
3  210000056.0   2022-12-02              2.0            2.0 2022-08-01   
4  210000057.0   2022-12-23              1.0            1.0 2019-08-01   

     unit material_description last_consumed_date  
0  Number     PACKING RING SET         2020-03-29  
1  Number         LANTERN RING         2020-03-29  
2     Set          PACKING SET         2020-03-29  
3  Number      LUG BRAKE PIVOT         2022-12-02 

In [184]:
from datetime import timedelta

# Aggregate to monthly with units
merged_df['month'] = merged_df['posting_date'].dt.to_period('M').dt.to_timestamp()
monthly_usage = merged_df.groupby(['material', 'month', 'unit'])['quantity_issued'].sum().reset_index(name='y')
monthly_usage['ds'] = monthly_usage['month']

# Add synthetic if needed (with unit)
def add_synthetic_if_needed(group):
    if len(group) < 2:
        last_date = group['ds'].min() - timedelta(days=30) if not group.empty else pd.Timestamp.today()
        unit = group['unit'].iloc[0] if not group.empty else 'Unknown'
        synthetic_row = pd.DataFrame({
            'ds': [last_date], 
            'y': [0], 
            'material': [group['material'].iloc[0]] if not group.empty else [None], 
            'unit': [unit],
            'month': [last_date]
        })
        group = pd.concat([group, synthetic_row])
    return group

monthly_groups = monthly_usage.groupby('material').apply(add_synthetic_if_needed).reset_index(drop=True)
monthly_groups['y'] = monthly_groups['y'].abs()
monthly_groups['month'] = monthly_groups['month'].fillna(monthly_groups['ds'])

# Aggregate plant usage per material using raw plant codes
plant_usage = consumption_df.groupby(['material', 'plant'])['quantity_issued'].sum().reset_index()
plant_usage['plant_summary'] = plant_usage['plant'].astype(str) + ': ' + plant_usage['quantity_issued'].astype(str) + ' units'
plant_usage_summary = plant_usage.groupby('material')['plant_summary'].apply(lambda x: ', '.join(x)).reset_index()

# Merge plant summary into monthly_groups (for forecasting context)
monthly_groups = monthly_groups.merge(plant_usage_summary, on='material', how='left')

print("\n✅ Fixed Monthly Aggregated Data Preview with Units and Plant Summary:")
print(monthly_groups.head())

print("\n✅ Monthly Data with Units and Plant Ready for ML")



✅ Fixed Monthly Aggregated Data Preview with Units and Plant Summary:
      material      month    unit    y         ds    plant_summary
0  210000050.0 2020-03-01  Number  1.0 2020-03-01  20D1: 1.0 units
1  210000050.0 2020-01-31  Number  0.0 2020-01-31  20D1: 1.0 units
2  210000051.0 2020-03-01  Number  1.0 2020-03-01  20D1: 1.0 units
3  210000051.0 2020-01-31  Number  0.0 2020-01-31  20D1: 1.0 units
4  210000052.0 2020-03-01     Set  1.0 2020-03-01  20D1: 1.0 units

✅ Monthly Data with Units and Plant Ready for ML


In [186]:
# Fix negative values (absolute for consumption focus)
monthly_groups['y'] = monthly_groups['y'].abs()

# Fix NaT in 'month' by syncing with 'ds'
monthly_groups['month'] = monthly_groups['month'].fillna(monthly_groups['ds'])

# Re-preview after fixes
print("\n✅ Fixed Monthly Aggregated Data Preview:")
print(monthly_groups.head())



✅ Fixed Monthly Aggregated Data Preview:
      material      month    unit    y         ds    plant_summary
0  210000050.0 2020-03-01  Number  1.0 2020-03-01  20D1: 1.0 units
1  210000050.0 2020-01-31  Number  0.0 2020-01-31  20D1: 1.0 units
2  210000051.0 2020-03-01  Number  1.0 2020-03-01  20D1: 1.0 units
3  210000051.0 2020-01-31  Number  0.0 2020-01-31  20D1: 1.0 units
4  210000052.0 2020-03-01     Set  1.0 2020-03-01  20D1: 1.0 units


In [188]:
forecast_file = 'forecast_results.pkl'
if os.path.exists(forecast_file):
    with open(forecast_file, 'rb') as f:
        forecast_results = pickle.load(f)
    print("✅ Loaded saved forecasts!")
else:
    print("🚀 Retraining with units...")
    
    top_n = 6703
    material_usage = monthly_groups.groupby('material')['y'].sum().sort_values(ascending=False).head(top_n)
    top_materials = material_usage.index.tolist()

    forecast_results = {}
    for i, mat in enumerate(top_materials):
        mat_df = monthly_groups[monthly_groups['material'] == mat][['ds', 'y']].dropna()
        unit = monthly_groups[monthly_groups['material'] == mat]['unit'].iloc[0] if not monthly_groups.empty else 'Unknown'
        
        if len(mat_df) < 2:
            continue
        
        model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
        model.fit(mat_df)
        
        future = model.make_future_dataframe(periods=6, freq='M')
        forecast = model.predict(future)
        
        today = pd.Timestamp.today()
        forecast_future = forecast[forecast['ds'] >= today]
        forecast_6m = forecast_future['yhat'].sum() if len(forecast_future) > 0 else 0
        
        forecast_results[mat] = {'forecast_6m': round(forecast_6m, 2), 'unit': unit}
        
        if (i + 1) % 100 == 0:
            print(f"✅ Processed {i + 1}/{len(top_materials)} materials...")

    with open(forecast_file, 'wb') as f:
        pickle.dump(forecast_results, f)
    print("✅ Forecasts saved!")

# Sample with units
print("\n✅ Sample 6-Month Forecasts with Units:")
for mat, vals in list(forecast_results.items())[:5]:
    print(f"Material {mat}: {vals['forecast_6m']} {vals['unit']}")


✅ Loaded saved forecasts!

✅ Sample 6-Month Forecasts with Units:
Material 220002929.0: 524.11 liters
Material 220002930.0: 0 liters
Material 220043166.0: -2175.85 liters
Material 220044435.0: 3019.49 liters
Material 252723168.0: 0 liters


In [196]:
from datetime import timedelta
import pandas as pd
import numpy as np

# --- Assumptions & Constants ---
PROCUREMENT_CYCLE_MONTHS = 6    # Procurement lead time (6 months, as per your norm)
TODAY = pd.Timestamp.today()    # Current date for reference calculations
THRESHOLD_PERCENT = 0.2         # Threshold fraction for reorder trigger (e.g. reorder if stock < 20% of some baseline)

# --- Step 4: Calculate Final Outputs ---

outputs = []

for mat in merged_df['material'].unique():
    mat_data = merged_df[merged_df['material'] == mat]
    # Get properly mapped unit for this material
    unit_rec = material_units[material_units['material'] == mat]['unit']
    unit = unit_rec.iloc[0] if not unit_rec.empty else 'Unknown'
    # Fetch 6-month forecast from saved model results
    forecasts = forecast_results.get(mat, {'forecast_6m': 0})
    forecast_6m = forecasts['forecast_6m']
    # Quantities with properly mapped unit
    forecast_6m_with_unit = f"{round(forecast_6m, 2)} {unit}"
    current_stock = mat_data['current_stock'].max() or 0
    reorder_threshold = max(current_stock * THRESHOLD_PERCENT, 1.0)
    if current_stock < reorder_threshold:
        risk_status = 'High - Reorder Now'
    else:
        risk_status = 'Low - OK'
    reorder_level_with_unit = f"{round(reorder_threshold, 2)} {unit}"
    # Next Material Availability
    last_stock_date = mat_data['stock_date'].max() or TODAY
    availability_date = last_stock_date + timedelta(days=PROCUREMENT_CYCLE_MONTHS * 30)
    next_availability = f"{availability_date.date()}: {round(current_stock, 2)} {unit}"
    plant_summary = plant_usage_summary[plant_usage_summary['material'] == mat]['plant_summary'].values[0] if mat in plant_usage_summary['material'].values else 'No Plant Data'

    outputs.append({
        'Material': mat,
        'Material Description': mat_data['material_description'].iloc[0] if 'material_description' in mat_data.columns else 'Unknown',
        'Plant Usage Summary': plant_summary,
        'Current Stock': f"{round(current_stock, 2)} {unit}",
        'Last Consumed Date': mat_data['last_consumed_date'].max().date() if 'last_consumed_date' in mat_data.columns else 'Unknown',
        'Next Material Availability': next_availability,
        'Forecasted Usage (6 Months)': forecast_6m_with_unit,
        'Reorder Level': reorder_level_with_unit,
        'Risk Status': risk_status
    })


# Create DataFrame and export
output_df = pd.DataFrame(outputs)

# Top 10 materials by usage with plant info
total_usage = consumption_df.groupby('material')['quantity_issued'].sum().reset_index()
top_10 = total_usage.nlargest(10, 'quantity_issued')
top_10 = top_10.merge(plant_usage_summary, on='material', how='left')
top_10.to_csv('top_materials_by_plant.csv', index=False)

print("\n✅ Updated Inventory Optimization Output (Sample):")
print(output_df.head(10))  # Show top 10 for verification

output_df.to_csv('updated_inventory_results.csv', index=False)
print("\n✅ Results exported to 'updated_inventory_results.csv'")



✅ Updated Inventory Optimization Output (Sample):
      Material Material Description Plant Usage Summary Current Stock  \
0  210000050.0     PACKING RING SET     20D1: 1.0 units    1.0 Number   
1  210000051.0         LANTERN RING     20D1: 1.0 units    1.0 Number   
2  210000052.0          PACKING SET     20D1: 1.0 units       1.0 Set   
3  210000056.0      LUG BRAKE PIVOT     20D1: 2.0 units    2.0 Number   
4  210000057.0   LUG-BRAKE ACTUATOR     20D1: 1.0 units    1.0 Number   
5  210000058.0            CAM-BRAKE     20D1: 1.0 units    1.0 Number   
6  210000060.0        BEAM-EQALIZER     20D1: 2.0 units    1.0 Number   
7  210000062.0       PIN-BRAKE BAND     20D1: 2.0 units    4.0 Number   
8  210000070.0             ROD PUSH    20E1: 12.0 units    nan Number   
9  210000071.0             ROD PUSH     20E1: 6.0 units    nan Number   

  Last Consumed Date Next Material Availability Forecasted Usage (6 Months)  \
0         2020-03-29     2020-04-29: 1.0 Number                   